In [ ]:
import os
os.chdir('/content/drive/MyDrive/data/real_data/')

In [ ]:
from src.utils import AbstractBaseModel

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import InputSpec
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

from tensorflow.keras.losses import kld

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [ ]:
class ClusteringLayer(Layer):
    
    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def ConvBNActBlock(x, activation, filter_size, filter_dim, kernel_initializer, kernel_regularizer, name=None):
    x = Conv2D(filters=filter_dim,
               kernel_size=filter_size,
               padding='same',
               kernel_initializer=kernel_initializer,
               kernel_regularizer=kernel_regularizer,
               name=f'{name}-conv')(x)
    x = BatchNormalization(name=f'{name}-bn')(x)
    x = Activation(activation, name=f'{name}-act')(x)
    
    return x

def ConvTrBNActBlock(x, activation, filter_size, filter_dim, kernel_initializer, kernel_regularizer, name=None):
    x = Conv2DTranspose(filters=filter_dim,
                        kernel_size=filter_size,
                        padding='same',
                        kernel_initializer=kernel_initializer,
                        kernel_regularizer=kernel_regularizer,
                        name=f'{name}-convtr')(x)
    x = BatchNormalization(name=f'{name}-bn')(x)
    x = Activation(activation, name=f'{name}-act')(x)
    
    return x

def BottleneckBlock(x, activation, embedding_dim, kernel_initializer, kernel_regularizer, name=None):
    _raw_shape = x.shape[1:]
    x = Flatten(name=f'{name}-flatten')(x)
    _flat_shape = x.shape[1]
    
    h = Dense(units=embedding_dim,
              kernel_initializer=kernel_initializer,
              kernel_regularizer=kernel_regularizer,
              name=f'{name}-embedding')(x)
    x = BatchNormalization(name=f'{name}-bn1')(h)
    x = Activation(activation, name=f'{name}-act1')(x)
    
    x = Dense(units=_flat_shape,
              kernel_initializer=kernel_initializer,
              kernel_regularizer=kernel_regularizer,
              name=f'{name}-dense')(x)
    x = Reshape(_raw_shape, name=f'{name}-reshape')(x)
    x = BatchNormalization(name=f'{name}-bn2')(x)
    x = Activation(activation, name=f'{name}-act2')(x)
    
    return h, x

In [ ]:
class DEC(object):
    def __init__(self, 
                 n_clusters='auto',
                 window_size=8, num_channels=3,
                 filter_size=20, filter_dim=50,
                 embedding_dim=64,
                 alpha=1.0,
                 kernel_initializer='glorot_normal',
                 kernel_regularizer=None):
        
        self._ae, self._enc = self._define_model(
                window_size, num_channels,
                filter_size, filter_dim,
                embedding_dim,
                kernel_initializer,
                kernel_regularizer)
        
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.pretrained = False
        
        if type(n_clusters) == int:
            cluster = ClusteringLayer(self.n_clusters, name='clustering')(self._enc.output)
            self.cluster = Model(self._enc.input, cluster)
        

    def train(self, x, maxiter=2000, batch_size=128, tol=1E-3,
              update_interval=250):

        if not self.pretrained:
            self.pretrain(x)
            
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self._enc.predict(x))
        y_pred_last = np.copy(y_pred)
        
        self.cluster.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
        sgd = keras.optimizers.SGD(lr = 0.001)
        self.cluster.compile(optimizer= sgd, loss='kld', metrics=['accuracy'])
        #self.cluster.compile(optimizer='SGD', loss='kld', metrics=['accuracy'], learning_rate=learning_rate)

        loss = 0
        index = 0
        index_array = np.arange(x.shape[0])
        for i in range(maxiter):
            if i % update_interval == 0:
                q = self.cluster.predict(x)
                p = self.target_distribution(q)

                y_pred = q.argmax(1)

                delta_label = np.mean(y_pred != y_pred_last).astype(np.float32)
                y_pred_last = np.copy(y_pred)
                
                if i > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    break
                
                np.random.shuffle(index_array)
                
            idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
            loss = self.cluster.train_on_batch(x=x[idx], y=p[idx])
            index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0


    def autofit(self, x, maxiter=2000, batch_size=128, learning_rate=1E-3, tol=1E-3,
                update_interval=200):
        
        train_x, valid_x = train_test_split(x, test_size=0.2)
        self._train_ae(train_x)
        
        n_clusters_range = list(range(2, 11))
        loss_ratio = []
        for n_clusters in n_clusters_range:
            cluster = ClusteringLayer(n_clusters, name='clustering')(self.enc.output)
            self.cluster = Model(self.enc.input, cluster)
            self.train(train_x, maxiter=maxiter, batch_size=batch_size, learning_rate=learning_rate,
                       tol=tol, update_interval=update_interval)
            
            train_q = self.cluster.predict(train_x)
            train_p = self.target_distribution(train_q)
            
            train_loss = kld(train_p, train_x)
            
            valid_q = self.cluster.predict(valid_x)
            valid_p = self.target_distribution(valid_q)
            
            valid_loss = kld(valid_p, valid_x)
            
            loss_ratio.append(train_loss / valid_loss)
        
        self.n_clusters = n_clusters_range[np.argmax(np.diff(loss_ratio)) + 1]
        cluster = ClusteringLayer(self.n_clusters, name='clustering')(self.enc.output)
        self.cluster = Model(self.enc.input, cluster)
        self.train(x)
        
    def predict(self, x):
        q = self.cluster.predict(x)
        return np.argmax(q, 1)
    
    def _get_embedding(self, x):
        z = self._enc.predict(x)
        return z
        
    def _get_recon_error(self, x):
        _x = self._ae.predict(x)
        recon_error = np.square(x - _x).mean((1, 2))
        return recon_error
    
    def pretrain(self, x, epochs=200, batch_size=128):
        self._ae.compile(optimizer='Adam', loss='mse', metrics=['mse'])
        self._ae.fit(x, x, batch_size=batch_size, epochs=epochs, callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, mode="min")
    ])
        self.pretrained = True

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T
    
    
    def _define_model(self, window_size, num_channels,
                      filter_size, filter_dim,
                      embedding_dim,
                      kernel_initializer,
                      kernel_regularizer,
                      activation='relu',
                      output_act=None):
        
        filter_size = (filter_size, 1)
        
        conv_block_args = {
                'activation': activation,
                'filter_size': filter_size,
                'kernel_initializer': kernel_initializer,
                'kernel_regularizer': kernel_regularizer
                }
        
        _x_in = Input(shape=(window_size, 1, num_channels), name='input')
        
        x = ConvBNActBlock(_x_in, name='block1', filter_dim=filter_dim, **conv_block_args)
        x = AveragePooling2D(pool_size=(2, 1), name='pool1')(x)
        
        x = ConvBNActBlock(x, name='block2', filter_dim=filter_dim*2, **conv_block_args)
        x = AveragePooling2D(pool_size=(2, 1), name='pool2')(x)
        
        x = ConvBNActBlock(x, name='block3', filter_dim=filter_dim*4, **conv_block_args)
        
        _z_out, x = BottleneckBlock(x, activation, embedding_dim, kernel_initializer,
                                    kernel_regularizer, name='blottleneck')
        
        x = UpSampling2D(size=(2, 1), name='upsample1')(x)
        x = ConvTrBNActBlock(x, name='block4', filter_dim=filter_dim*2, **conv_block_args)
        
        x = UpSampling2D(size=(2, 1), name='upsample2')(x)
        x = ConvTrBNActBlock(x, name='block5', filter_dim=filter_dim, **conv_block_args)
        
        x = Conv2DTranspose(filters=num_channels, kernel_size=filter_size, padding='same',
                            kernel_initializer=kernel_initializer,
                            kernel_regularizer=kernel_regularizer,
                            name='output-convtr')(x)
        _x_out = Activation(activation=output_act, name='output-act')(x)
        
        ae_model = Model(inputs=_x_in, outputs = _x_out, name='AutoencoderModel')
        enc_model = Model(inputs=_x_in, outputs = _z_out, name='EncoderModel')
        return ae_model, enc_model

In [ ]:
# 프로젝트 경로 설정 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import sys

project_dir = "/content/drive/MyDrive/data/"

project_dir = os.path.abspath(project_dir)

if project_dir not in sys.path:

    sys.path.append(project_dir)

data_dir = os.path.join(project_dir, 'real_data/')
# 경로를 합친다 ==> /content/drive/MyDrive/data/real_data/ + data/
save_dir = os.path.join(project_dir, 'real_save/')

# os.makedirs(data_dir, exist_ok=True)
# os.makedirs  ==> 파일 Dir 생성
# os.makedirs(save_dir, exist_ok=True)
# os.makedirs  ==> 파일 Dir 생성

In [ ]:
# 데이터

data_filenames = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
# os.listdir ==> 디렉토리 내부에 파일 이름을 반환한다.

data_filenames

df = pd.read_csv(data_dir+'303_1008.csv')

df = df.sort_values(by='Time')

df

In [ ]:
# 데이터 타입 및 변수 정의

# types = pd.DataFrame(df.dtypes, columns=['type'])

# obj_var = list(types[types['type']=='object'].index)

# num_var = list(set(df.columns) - set(obj_var))

# target = df['Score']

# df['Date'] = [_d.split(' ')[0] for _d in df['Date']]

In [ ]:
# 사용할 컬럼 
sensor = ['X','Y','Z']
# 사용하지 않을 컬럼
# drop_col = ['time']

In [ ]:
# def process_window(signal, label, size, slide=None):

def process_window(signal, size, slide=None):
    # df_ID_date[sensor].values /  df_ID_date['y'].values / 8 / none

    # assert len(signal) == len(label)

    if slide is None:

        slide = 4
        # size =  8
        

    # x, y = [], []
    x = []

    for start in range(0, len(signal)-size, slide):

        end = start + size

        x.append(signal[start:end])

        # y.append(label[start:end].any() + 0)

    # return np.array(x), np.array(y)
    return np.array(x)

In [ ]:
X= []

scale_col = ['X','Y','Z']

std = StandardScaler()

df[scale_col] = std.fit_transform(df[scale_col].values)

df[scale_col] = pd.DataFrame(df[scale_col],index=df[scale_col].index, columns=df[scale_col].columns)
# sensor = ['X','Y','Z']

In [ ]:
df

In [ ]:
# # for ID in df['time'].unique():

# #     df_ID = df[df['time'] == time]

# for date in df['Time'].unique():

#     df_ID_date = df
#     _X = process_window(df_ID_date[sensor].values, 100)

#     if len(_X) > 0:

#         X.append(_X)
     
#         # y.append(_y)

# X = np.row_stack(X)
# # y = np.concatenate(y)

In [ ]:
df_ID_date = df
_X = process_window(df_ID_date[sensor].values, 8)

if len(_X) > 0:
    X.append(_X)

X = np.row_stack(X)

In [ ]:
X[0]

In [ ]:
X.shape
x = np.expand_dims(X, 2)
x.shape

In [ ]:

#np.unique(y)


# model fit
# n_cluster cluster 개수 지정

dec = DEC(n_clusters=8)
dec.train(x)

a = dec.predict(x)


In [ ]:
kmean_df = pd.DataFrame(a)
kmean_df

x = kmean_df[0].unique()
y = kmean_df[0].value_counts()
y = y.sort_index()

plt.bar(x, y, width=0.5)

for i, v in enumerate(x):
    plt.text(v, y[i], y[i],                 # 좌표 (x축 = v, y축 = y[0]..y[1], 표시 = y[0]..y[1])
             fontsize = 9, 
             color='black',
             horizontalalignment='center',  # horizontalalignment (left, center, right)
             verticalalignment='bottom')    # verticalalignment (top, center, bottom)

In [ ]:
import matplotlib.pyplot as plt
np.unique(a, return_counts=True)
# bb = confusion_matrix(y, a)

scores_mean = []
scores_max = []
for c in np.unique(a):
    x_c = x[a == c]
    scores_mean.append(dec._get_recon_error(x_c).mean(1).mean())
    scores_max.append(dec._get_recon_error(x_c).max(1).mean())

In [ ]:
scores_max

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_title('DEC Results')
ax1.set_xlabel('Cluster')
ax1.set_ylabel('Average Reconstruction Error')
ax1.plot(scores_mean, marker='x')

ax2 = ax1.twinx()
ax2.set_ylabel('Number of Anomalies')
ax2.plot(bb[1], marker='x', color='red')
plt.show()

ax2 = ax1.twinx()
ax2.set_ylabel('Number of Anomalies')
ax2.plot(bb[1], marker='x', color='red')
plt.show()
print(bb[:2])

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_title('DEC Results')
ax1.set_xlabel('Cluster')
ax1.set_ylabel('Max Channel-Reconstruction Error')
ax1.plot(scores_max, marker='x')